# Running the CUES SUMMA setup

First, we have to install the setup, which simply fixes paths in the file manager.

In [ ]:
! cd /home/jovyan/cues/summa_setup && ./install_local_setup.sh

In [ ]:
%pylab inline
import pysumma as ps
import xarray as xr

<br>

## Instantiating a simulation object

To set up a `Simulation` object you must supply 2 pieces of information. 

First, the SUMMA executable; this could be either the compiled executable on your local machine, or a docker image. 
For this case, I'll assume that `summa.exe` is on your path. 
See the commented out `executable` for example of how you could also define the docker image. 
The string for the docker image simply came from looking at the output of the `docker images` command.

The second piece of information is the path to the file manager, which we just created through the install script.
To create the `Simulation` object you can then just pass these to the constructor as shown below.

In [ ]:
executable = 'summa.exe'
file_manager = '/home/jovyan/cues/summa_setup/file_manager.txt'

s = ps.Simulation(executable, file_manager)

<br>

## Manipulating the configuration of the simulation object

Most of your interactions with pysumma will be facilitated through this `Simulation` object, so let's take some time to look through what is in it. 
What's contained in the `Simulation` object right after instantiation is generally just the input required for a SUMMA run.
For a more in depth discussion of what these are see the [SUMMA Input](https://summa.readthedocs.io/en/latest/input_output/SUMMA_input/) page of the documentation.
There are several attributes of interest that can be examined. 
To see each of them you can simply `print` them. 
Here's a very high level overview of what's available:

* `s.manager` - the file manager
* `s.decisions` - the decisions file
* `s.output_control` - defines what variables to write out
* `s.force_file_list` - a listing of all of the forcing files to use
* `s.local_attributes` - describes GRU/HRU attributes (lat, lon, elevation, etc)
* `s.local_param_info` - listing of spatially constant local (HRU) parameter values
* `s.basin_param_info` - listing of spatially constant basin (GRU) parameter values
* `s.parameter_trial` - spatially distributed parameter values (will overwrite `local_param_info` values, can be either HRU or GRU)

Most of these objects have a similar interface defined, with exceptions being `local_attributes` and `parameter_trial`. Those two are standard `xarray` datasets. All others follow the simple API:

```
print(x)                   # Show the data as SUMMA reads it
x.get_option(NAME)         # Get an option
x.set_option(NAME, VALUE)  # Change an option
x.remove_option(NAME)      # Remove an option
```

More intuitively, you can use `key` - `value` type indexing like dictionaries, dataframes, and datasets:

```
print(x['key'])    # Get an option
x['key'] = value   # Change an option
```

### Setting decisions

So, now that we've got a handle on what's available and what you can do with it, let's actually try some of this out. First let's just print out our decisions file so we can see what's in the defaults.


In [ ]:
print(s.decisions)

<br> 

Great, we can see what's in there. But to be able to change anything we need to know the available options for each decision. Let's look at how to do that. For arbitrary reasons we will look at the `snowIncept` option, which describes the parameterization for snow interception in the canopy. First we will get it from the `decisions` object directly, and then query what it can be changed to, then finally change the value to something else.

<br>

In [ ]:
# Get just the `snowIncept` option
print(s.decisions['snowIncept'])

# Look at what we can set it to
print(s.decisions['snowIncept'].available_options)

# Change the value 
s.decisions['snowIncept'] = 'stickySnow'
print(s.decisions['snowIncept'])

<br>

### Changing parameters

Much like the decisions we can manipulate the `local_param_info` file. First, let's look at what's contained in it

In [ ]:
print(s.local_param_info)

<br>

<br>

Yikes, that's pretty long. Okay but we can change things. See:

In [ ]:
# Print it
print(s.local_param_info['albedoMax'])

# Change the value
s.local_param_info['albedoMax'] = 0.9
print(s.local_param_info['albedoMax'])

# Change the value again - the right two values are currently unused by SUMMA though so this is superfluous
s.local_param_info['albedoMax'] = [0.9, 0.2, 1]
print(s.local_param_info['albedoMax'])

<br>

### Modifying output
And one more, we can also modify what get's written to output. 
The output control file represents the options available through columns of numeric values.
These numbers represent how to write the output. 
From the SUMMA documentation they are arranged as:

```
! varName          | outFreq | inst | sum | mean | var | min | max | mode
```

As before, let's look at what's in the `output_control` by simply printing it out.

<br>

In [ ]:
print(s.output_control)

<br>

Then, we can modify values in a couple of ways.

<br>

In [ ]:
# Just change the frequency to daily output
print(s.output_control['scalarNetRadiation'])
print(s.output_control['scalarNetRadiation'].statistic)

# Change the output statistic from instantaneous to sum
s.output_control['scalarNetRadiation'] = [1, 1, 0, 0, 0, 0, 0, 0]
s.output_control['scalarSnowAge'] = [1, 1, 0, 0, 0, 0, 0, 0]
print(s.output_control['scalarNetRadiation'])
print(s.output_control['scalarNetRadiation'].statistic)

# We could also be more verbose:
s.output_control['scalarNetRadiation'] = {
    'period': 1, 'instant': 1, 'sum': 0, 
    'mean': 0, 'variance': 0, 'min': 0, 'max': 0
}
print(s.output_control['scalarNetRadiation'])
print(s.output_control['scalarNetRadiation'].statistic)

<br>

## Running pysumma and manipulating output

Now that you've had an overview of how you can interact with SUMMA configurations through pysumma let's run a simulation. 
Before doing so we will reset our `Simulation` object, which will discard all of the changes we've made and load in a clean setup. 
Alternatively you could simply instantiate a new `Simulation` object.
After running the simulation, we will make sure that it completed successfully by checking the status.
With a complete run, we can look at the output simply by using the simulation's `output` attribute.
It is simply an xarray dataset, which can be manipulated in all of the usual ways.

In [ ]:
s.reset()
# Or you could just create a new simulation object like before:
#s = ps.Simulation(executable, file_manager)

s.run('local', run_suffix='_default')
assert s.status == 'Success'

In [ ]:
s.output['scalarSWE'].plot(label='SUMMA')
plt.legend()

## Changing the snow layering

In this example we will change the snow layering scheme so that only a maximum of 2 layers will ever exist. We do this by making it so that the amount of snow needed to split layers is very large. We will also make sure we are using the `CLM_2010` snow layering scheme.

<div class="alert alert-block alert-warning" style="background: lemonchiffon; padding: 8px">
<b>Note:</b> 
    Once you have started the run with `s_2layer.run(...)` the settings chosen will be written to disk. You can restore the default settings by checking out the original setup in git with `git reset --hard`. Alternatively, you may just want to back things up locally.
</div>


In [ ]:
s_2layer = ps.Simulation(executable, file_manager)
s_2layer.decisions['snowLayers']  = 'CLM_2010'

In [ ]:
s_2layer.local_param_info['zminLayer3'] = [100., 0.05, 0.05]
s_2layer.local_param_info['zminLayer4'] = [100., 0.1, 0.1]
s_2layer.local_param_info['zminLayer5'] = [100., 0.25, 0.25]
s_2layer.local_param_info['zmaxLayer2_lower'] = [9000., 0.2, 0.2]
s_2layer.local_param_info['zmaxLayer3_lower'] = [9000., 0.5, 0.5]
s_2layer.local_param_info['zmaxLayer4_lower'] = [9000., 1., 1.]
s_2layer.local_param_info['zmaxLayer2_upper'] = [9000., 0.15, 0.15]
s_2layer.local_param_info['zmaxLayer3_upper'] = [9000., 0.3, 0.3]
s_2layer.local_param_info['zmaxLayer4_upper'] = [9000., 0.75, 0.75]

In [ ]:
s_2layer.run('local', run_suffix='_2layer')
assert s_2layer.status == 'Success'

In [ ]:
s_2layer.output['scalarSWE'].plot(label='SUMMA 2 Layer')
s.output['scalarSWE'].plot(label='SUMMA default')
plt.legend()